# Genetic Algorithm and Cellular Automata

## Genetic Algorithm

In [54]:
import ioh
import random
import numpy as np
from algorithm import Algorithm

class GeneticAlgorithm(Algorithm):



    def __init__(self, pop_size=10, dim=10, seed=10, mutation_type = "swap"):
        super().__init__(max_iterations=1000)
        self.mutation_type = mutation_type
        self.dim = dim
        self.seed = seed
        self.dim = dim
        self.population_size = pop_size
        np.random.seed(self.seed)
        self.population = self.generate_population()
        self.y_best = 0.0
        self.x_best : list[int] = []


    def generate_candidate(self):
        """
        Generate an candidate i for the population self.population
        """
        candidate = np.random.randint(2, size=self.dim, dtype=int)
        return candidate

    
    def generate_population(self):
        """
        Generate a candidate for every i in self.population_size
        """
        return [self.generate_candidate() for i in range(self.population_size)]

    
    def print_pop_size(self):
        """
        Print the population size
        """
        print(self.population_size)

       
    def selection(self, k=2):
        """
        * Calculate fitness score/weight for every candidate
        * Randomly select k candidates given selection probabilities
        * return a list of two candidates
        """
        candidates = np.empty(shape=[0, 2])

        for candidate in self.population:
            candidates = np.append(candidates, [[candidate, self.selection_probability(candidate)]], axis=0)

        weights = np.array(candidates[:,1], dtype=float)
        np_candidates = np.array(candidates[:,0])

        return np.random.choice(np_candidates, p = weights, size = 2)

        
    def selection_probability(self, candidate):
        """
        Return candidate fitness score / population fitness score
        """
        return (self.fitness(candidate) / self.population_fitness())

        
    def mutation(self, offspring):
        """
        For every child in the offspring apply either a swap mutatation or an insert mutation
        """
        if self.mutation_type == "swap":
            for child in offspring:
                child = self.swap_mutation(child)
        else:
            for child in offspring:
                child = self.insert_mutation(child)
        return offspring

   
    def swap_mutation(self, child):
        """
        Apply a swap mutation, by randomly selecting 2 genes and swapping them.
        """
        rand_idx = np.random.randint(self.dim,size=2)
        temp = child[rand_idx[1]]
        child[rand_idx[1]] = child[rand_idx[0]]
        child[rand_idx[0]] = temp
        return child

    
#     def insert_mutation(self, child):
#         """
#         * Apply an insert mutation, by randomly selecting an i and a j in the genome.
#         * Insert j at position i+1 and delete it's previous index
#         """
#         print('before mut:', child)
#         child = list(child)
#         rand_idx = np.random.randint(self.dim,size=2)
#         while rand_idx[0] > rand_idx[1]:
#             rand_idx = np.random.randint(self.dim,size=2)
#         if rand_idx[0] != rand_idx[1]:
#             child.insert(rand_idx[1]-1, child[rand_idx[0]])
#             child.pop(rand_idx[0])
#         print('after mut:', child)
#         return np.array(child, dtype=int)
    
    def insert_mutation(self, child):
        """
        * Apply an insert mutation, by randomly selecting an i and a j in the genome.
        * Insert j at position i+1 and delete it's previous index
        """
        child = list(child)
        i_idx = np.random.randint(self.dim-1)
        j_idx = np.random.randint(self.dim)
        if i_idx != j_idx:
            child.insert(i_idx+1, child[j_idx])
            child.pop(j_idx)
        return np.array(child, dtype=int)



    def crossover(self, parents):
        """
        For every child in the offspring apply a swap mutation.
        """
        if(len(parents) != 2):
            raise ValueError("There should be 2 parents")
        offspring = []
        for i in range(self.population_size):
            offspring.append(self.crossover_operator(parents))

        return offspring

    
    def crossover_operator(self, parents):
        """
        * Given two parents, create a child based on a random uniform probability
        * Children are created by a simple treshold. If the probability of a certain index in idx_prob is higher than 0.5, 
        * then a gene from parent A is given, else: parent B
        * This is done for n = self.population_size 
        """
        parentA = np.array(parents[0], dtype=int)
        parentB = np.array(parents[1], dtype=int)
        child = np.empty(shape=[0,1], dtype=int)
        idx_prob = np.random.random_sample(self.dim)
        for i in range(len(idx_prob)):
            if idx_prob[i] > 0.5:
                child = np.append(child, parentA[i])
            else:
                child = np.append(child, parentB[i])
        return child


    
    def fitness(self, candidate):
        """   
        Return the occurrences of 1s
        """
        return np.count_nonzero(candidate)
#         return ioh.problem(candidate)

    
    def population_fitness(self):
        """"
        Calculate the fitness score for the whole population
        Calculation of the fitness score is explained in fitness(self, candidate)
        """
        pop_fitness = 0
        for candidate in self.population:
            pop_fitness += self.fitness(candidate)
        return pop_fitness

    def __call__(self, problem: ioh.problem.Integer) -> None:
        self.dim = problem.meta_data.n_variables
        self.population = self.generate_population()
        while (problem.state.evaluations < self.max_iterations) and (self.x_best != problem.objective.x):
            parents = self.selection(self.population)
            offspring = self.crossover(parents)
            mutated_offspring = self.mutation(offspring)
            for candidate in self.population:
#                 new_y = self.fitness(candidate)
                new_y = problem(candidate)
                if new_y > self.y_best:
                    self.y_best = new_y
                    self.x_best = list(candidate)
                    print('best x: ', self.x_best)
            self.population = mutated_offspring

        problem(self.x_best)
        print('evaluations: ', problem.state.evaluations)
        return problem.state.current_best





In [55]:
def main():

    # Set a random seed in order to get reproducible results
    random.seed(42)

    # Get a problem from the IOHexperimenter environment
    problem: ioh.problem.Integer = ioh.get_problem(1, 1, 5, "Integer")

    # Instantiate the algoritm, you should replace this with your GA implementation
    algorithm = GeneticAlgorithm(mutation_type="insert")

    # Run the algoritm on the problem
    algorithm(problem)

    # Inspect the results
    print("Best solution found:")
    print("".join(map(str, problem.state.current_best.x)))
    print("With an objective value of:", problem.state.current_best.y)
    print()
    
main()

best x:  [1, 0, 0, 1, 0]
best x:  [1, 1, 1, 0, 1]
best x:  [1, 1, 1, 1, 1]
evaluations:  41
Best solution found:
11111
With an objective value of: 5.0



## Cellular Automata and the inversing problem:

In [57]:
import typing
import ioh

from implementation import RandomSearch

class CellularAutomata:
    '''Skeleton CA, you should implement this.'''
    
    def __init__(self, rule_number: int):
        super().__init__(max_iterations=1000)
        

        pass

    def __call__(self, c0: typing.List[int], t: int) -> typing.List[int]:
        '''Evaluate for T timesteps. Return Ct for a given C0.'''
        pass

In [27]:
a = np.random.normal(loc=4.5,size=8)
a /= a.sum()
a = np.round(a,2)
a

array([0.18, 0.15, 0.12, 0.13, 0.12, 0.08, 0.09, 0.12])